# 变分自动编码器
变分编码器是自动编码器的升级版本，其结构跟自动编码器是类似的，也由编码器和解码器构成。

回忆一下，自动编码器有个问题，就是并不能任意生成图片，因为我们没有办法自己去构造隐藏向量，需要通过一张图片输入编码我们才知道得到的隐含向量是什么，这时我们就可以通过变分自动编码器来解决这个问题。

其实原理特别简单，只需要在编码过程给它增加一些限制，迫使其生成的隐含向量能够粗略的遵循一个标准正态分布，这就是其与一般的自动编码器最大的不同。

这样我们生成一张新图片就很简单了，我们只需要给它一个标准正态分布的随机隐含向量，这样通过解码器就能够生成我们想要的图片，而不需要给它一张原始图片先编码。

一般来讲，我们通过 encoder 得到的隐含向量并不是一个标准的正态分布，为了衡量两种分布的相似程度，我们使用 KL divergence，利用其来表示隐含向量与标准正态分布之间差异的 loss，另外一个 loss 仍然使用生成图片与原图片的均方误差来表示。

KL divergence 的公式如下

$$
D{KL} (P || Q) =  \int_{-\infty}^{\infty} p(x) \log \frac{p(x)}{q(x)} dx
$$

## 重参数
为了避免计算 KL divergence 中的积分，我们使用重参数的技巧，不是每次产生一个隐含向量，而是生成两个向量，一个表示均值，一个表示标准差，这里我们默认编码之后的隐含向量服从一个正态分布的之后，就可以用一个标准正态分布先乘上标准差再加上均值来合成这个正态分布，最后 loss 就是希望这个生成的正态分布能够符合一个标准正态分布，也就是希望均值为 0，方差为 1

所以标准的变分自动编码器如下

![](https://ws4.sinaimg.cn/large/006tKfTcgy1fn15cq6n7pj30k007t0sv.jpg)

所以最后我们可以将我们的 loss 定义为下面的函数，由均方误差和 KL divergence 求和得到一个总的 loss

```
def loss_fun(recon_x, x, mean, std, eps=1e-8):
    """
    recon_x: generating images
    x: original images
    mean: latent mean
    var: latent var
    """
    mse = tf.reduce_sum(tf.square(x - recon_x))
    # 0.5 * sum(mu^2 + std^2 - 2log(std) - 1)
    kld_element = tf.square(mean) + tf.square(std) - 2.0 * tf.log(std + eps) - 1
    kld = 0.5 * tf.reduce_sum(kld_element)
    
    return mse + kld
```

下面我们用 mnist 数据集来简单说明一下变分自动编码器

In [ ]:
from __future__ import division
from __future__ import absolute_import
from __future__ import print_function

import numpy as np

import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.examples.tutorials.mnist.input_data as input_data

tf.set_random_seed(2017)

In [ ]:
mnist = input_data.read_data_sets('MNIST_data')
train_set = mnist.train
test_set = mnist.test

In [ ]:
input_ph = tf.placeholder(tf.float32, shape=[None, 784])
inputs = tf.divide(input_ph - 0.5, 0.5)

In [ ]:
def vae(inputs, scope='vae', reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        with slim.arg_scope([slim.fully_connected], activation_fn = tf.nn.relu):
            # 编码
            with tf.variable_scope('encoder'):
                encode = slim.fully_connected(inputs, 400, scope='fc1')
                mean = slim.fully_connected(encode, 20, activation_fn=None, scope='fc2_mean')
                logvar = slim.fully_connected(encode, 20, activation_fn=None, scope='fc2_var')
                
            # 重新参数化成正态分布
            with tf.variable_scope('reparametrize'):
                std = tf.sqrt(tf.exp(logvar))
                eps = tf.random_normal([20,], name='epsilon')
                rep = eps * std + mean
                
            # 解码
            with tf.variable_scope('decoder'):
                decode = slim.fully_connected(rep, 400, scope='fc3')
                decode = slim.fully_connected(decode, 784, activation_fn=tf.tanh, scope='fc4')
                
            return decode, mean, std

In [ ]:
outputs, mean, std = vae(inputs)

In [ ]:
def loss_fun(recon_x, x, mean, std, eps=1e-8):
    """
    recon_x: generating images
    x: original images
    mean: latent mean
    var: latent var
    """
    # mse
    mse = tf.reduce_sum(tf.square(x - recon_x))
    
    # kl divergence
    kld_element = tf.square(mean) + tf.square(std) - 2.0 * tf.log(std + eps) - 1
    kld = 0.5 * tf.reduce_sum(kld_element)
    
    return mse + kld

In [ ]:
loss = loss_fun(outputs, inputs, mean, std)

opt = tf.train.AdamOptimizer(1e-3)
train_op = opt.minimize(loss)

In [ ]:
gt = tf.reshape(input_ph, (-1, 28, 28, 1))
pre = tf.reshape(outputs, (-1, 28, 28, 1)) * 0.5 + 0.5
images = tf.concat([gt, pre], axis=2)[:3]
images_sum = tf.summary.image('images', images)

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(tf.global_variables_initializer())

train_writer = tf.summary.FileWriter('vae/train', sess.graph)
val_writer = tf.summary.FileWriter('vae/val', sess.graph)

In [ ]:
for e in range(100):
    num_examples = 0
    while num_examples < train_set.num_examples:
        if num_examples + 128 < train_set.num_examples:
            batch = 128
        else:
            batch = train_set.num_examples - num_examples
        num_examples += batch
        train_imgs, _ = train_set.next_batch(batch)
        sess.run(train_op, feed_dict={input_ph: train_imgs})
    if (e + 1) % 20 == 0:
        train_imgs_sum, train_loss = sess.run([images_sum, loss], feed_dict={input_ph: train_imgs})
        train_writer.add_summary(train_imgs_sum)
        val_imgs, _ = test_set.next_batch(128)
        val_imgs_sum, val_loss = sess.run([images_sum, loss], feed_dict={input_ph: val_imgs})
        val_writer.add_summary(val_imgs_sum)
        print('Epoch: {}, train_loss: {:.5f}, val_loss: {:.5f}'.format(e + 1, train_loss, val_loss))
train_writer.close()
val_writer.close()

可以看看使用变分自动编码器得到的结果，可以发现效果比一般的编码器要好很多

<img src="https://image.ibb.co/dY0GtH/variant_autoencoder.png">

我们可以输出其中的均值看看

In [ ]:
imgs, _ = train_set.next_batch(1)
mean_value = sess.run(mean, feed_dict={input_ph: imgs})
print(mean_value)

变分自动编码器虽然比一般的自动编码器效果要好，而且也限制了其输出的编码 (code) 的概率分布，但是它仍然是通过直接计算生成图片和原始图片的均方误差来生成 loss，这个方式并不好，在下一章生成对抗网络中，我们会讲一讲这种方式计算 loss 的局限性，然后会介绍一种新的训练办法，就是通过生成对抗的训练方式来训练网络而不是直接比较两张图片的每个像素点的均方误差